## Summary: Edelweiss C and its Error

This notebook is a copy of edelweiss_C_systematicErrors_allParameters.ipynb, but using more accurate calculations for the NR and ER bands.

## Band Widths
Edelweiss parametrizes their band widths for all detectors (see eq'ns 9, 10, and 11 from the above paper).  

$$\sigma_{Q\gamma}(E_R) = \frac{1 + V/\epsilon}{E_R}\sqrt{\sigma_I^2 + \sigma_H^2}$$

$$\sigma_{Qn}(E_R) = \sqrt{C^2 + \frac{1}{E^2_R} \left( \left(1+\frac{V}{\epsilon}\langle Q_n\rangle\right)^2\sigma_I^2 + \left( 1+\frac{V}{\epsilon}\right)^2\langle Q_n\rangle^2\sigma_H^2\right)} $$

where $C$ is a fitted constant; $\epsilon$ is the average energy needed to create a single electron-hole pair in germanium (3 eV); and $\langle Q_n \rangle$, $\sigma_I^2$, and $\sigma_H^2$ are all functions of the true recoil energy $E_R$. The yield $\langle Q_n \rangle = 0.16 E_R^{0.18}$.  The terms $\sigma_I^2$ and $\sigma_H^2$ should be read as "the heat/ionization resolution for an energy deposit of $E_R$."  For the electron recoil band and $\sigma_{Q\gamma}$, the arguments for$\sigma_I^2$ $\sigma_H^2$ are $E_R$ since for electron recoils, $E_I = E_R$ and $E_H = E_R$.  For nuclear recoils the heat and ionization energy are not identical to the energy $E_R$ and are defined in the section below.

In [1]:
import numpy as np
import pandas as pd
from joblib import Memory
import matplotlib.pyplot as plt
%matplotlib inline 
import h5py
from scipy import optimize
import lmfit as lmf
import emcee

import sys
sys.path.append('../python/')
from EdwRes import *
from prob_dist import *

In [2]:
# GGA3 parameters from Edelweiss tables
ion_center_0keV = 1.3
ion_guard_0keV = 1.5
heat_0keV = 0.4
ion_122keV = 3.1 
heat_122keV = 2.7
aH = 0.0157
pars = {
    'V' : 4.0,
    'eps_eV' : 3.0
    }

In [3]:
from edelweiss_fit import *

# import data from Edelweiss
resNR_data = pd.read_csv("data/edelweiss_NRwidth_GGA3_data.txt", skiprows=1, \
                       names=['E_recoil', 'sig_NR', 'E_recoil_err', 'sig_NR_err'], \
                       delim_whitespace=True)

resER_data = pd.read_csv("data/edelweiss_ERwidth_GGA3_data.txt", skiprows=1, \
                         names=['E_recoil', 'sig_ER', 'sig_ER_err'], \
                         delim_whitespace=True)

# the sorting is necessary!
# otherwise the mask defined below will select the wrong data
resER_data = resER_data.sort_values(by='E_recoil')

#print (res_data.head(4))

# set the data up for the fits
# Edelweiss discards ER points near peaks
# and first two NR points since they're affected by the threshold
mask = [True, True, False, False, True, True, True, True, True]
ER_data = {'Erecoil': resER_data["E_recoil"][mask], 'sigma': resER_data["sig_ER"][mask], 'sigma_err': resER_data["sig_ER_err"][mask]}
NR_data = {'Erecoil': resNR_data["E_recoil"][2::], 'sigma': resNR_data["sig_NR"][2::], 'sigma_err': resNR_data["sig_NR_err"][2::]}

## Fit with an MCMC method

In [33]:
def log_likelihood(theta, ER_data, NR_data):
    aH, C, m, scale, A, B = theta
    # Anthony's function use the FWHM version of aH
    aH = 2*np.sqrt(2*np.log(2))
    x_ER, y_ER, yerr_ER = ER_data['Erecoil'], ER_data['sigma'], ER_data['sigma_err']
    x_NR, y_NR, yerr_NR = NR_data['Erecoil'], NR_data['sigma'], NR_data['sigma_err']
    #x_yield, y_yield, yerr_yield = yield_data['Erecoil'], yield_data['Yield'], yield_data['Yield_err']
    
    # expected parameter values and widths
    # scale width estimated by assuming a 10 mV error on V and 0.5 eV error on epsilon
    # information for A and B from Astroparticle Physics 14 (2001) 329±337
    # uncertainty on aH is the uncertainty on the parameter aH when fitting only the ER band
    exp_aH = 0.016*2*np.sqrt(2*np.log(2))
    exp_aH_sig = exp_aH*0.046*2*np.sqrt(2*np.log(2))
    exp_scale = 1
    exp_scale_sig = 0.17
    exp_A = 0.16
    exp_A_sig = 0.07
    exp_B = 0.18
    exp_B_sig = 0.1
    
    # GGA3 parameters from Edelweiss tables
    ion_center_0keV = 1.3
    ion_guard_0keV = 1.5
    heat_0keV = 0.4
    ion_122keV = 3.1 
    heat_122keV = 2.7
    par_dict = {'V' : scale*4.0,'eps_eV' : 3.0, 'a': A, 'b': B}
    
    sigER_func = get_sig_gamma_func(ion_center_0keV, ion_guard_0keV, ion_122keV, heat_0keV, heat_122keV, \
                                    par_dict, aH)
    
    model_ER = sigER_func(x_ER)
    sigma2_ER = yerr_ER**2
    # series_NRQ_var_corr1(Er=10.0,F=0.0,V=4.0,aH=0.0381,alpha=(1/18.0),A=0.16,B=0.18,label='GGA3',corr1file='data/sigdiff_test.h5')
    # series_NRQ_var_corr1 returns the *variance*
    model_NR_0 = [series_NRQ_var_corr1(x, 0, par_dict['V'], aH, 1/18.0, A, B, 'GGA3') for x in x_NR] #sigNR_func(x_NR)
    model_NR = np.sqrt(np.power(C + m*x_NR, 2) + model_NR_0)
    sigma2_NR = yerr_NR**2
    """    
    model_yield = Q_avg(x_yield, A, B)
    sigma2_yield = yerr_yield**2
    """
    
    return -0.5*(np.sum((y_NR-model_NR)**2/sigma2_NR + np.log(2*np.pi*sigma2_NR)) \
                 + np.sum((y_ER-model_ER)**2/sigma2_ER + np.log(2*np.pi*sigma2_ER)) \
                 + (aH - exp_aH)**2/exp_aH_sig**2 + (scale - exp_scale)**2/exp_scale_sig**2 + (A - exp_A)**2/exp_A_sig**2 + (B - exp_B)**2/exp_B_sig**2)

In [34]:
def gauss_log_prior(theta):
    aH, C, m, scale, A, B = theta
    """    
    scale_sig_sq = 0.5*0.5
    A_sig_sq = (0.07)**2
    B_sig_sq = (0.1)**2
    scale_expected = 1
    A_exp = 0.16
    B_exp = 0.18
    """
    
    if -0.7 < aH < 0.7 and 0.02 < C < 0.05 and -1e-3 < m < 1e-3 and 0 < scale < 100 and 0 < A < 0.3 and 0 < B < 0.3:
        return 0.0 
        #return 0.0 - (scale-scale_expected)**2/(2*scale_sig_sq) - (A - A_exp)**2/(2*A_sig_sq) - (B - B_exp)**2/(2*B_sig_sq)
    return -np.inf

In [35]:
def new_log_probability(theta, ER_data, NR_data, prior_func):
    lp = prior_func(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, ER_data, NR_data)

## Find the minimum likelihood

When all six parameters are allowed to vary, the solution finds a small value for A and a larger value for B:

aH, C, m, scale, A, B = array([1.59557510e-02, 4.01120006e-02, 4.11442471e-05, 1.03493435e+00, 1.16847345e-01, 1.52032678e-01])

### Global minimization results
Brute: sadly this does not complete in a reasonable time (less than a day)

Basinhopping: does not successfully converge on a solution

SHGO: overflows occur in the yeild function but it still reports converging successfully with a function value of -53.623 and solution of xl: array([[1.59555935e-02, 4.01182258e-02, 4.10754863e-05, 1.03491851e+00,
        1.16818488e-01, 1.51974313e-01]]).


In [36]:
# if fit_test is false then the fit will be run
fit_test = False

# if mcmc_test is false then the MCMC sampler will be run
mcmc_test = True

In [37]:
if not fit_test:
    # start at about the minimum
    aH, C, m, scale, A, B = 0.0164*2*np.sqrt(2*np.log(2)), 0.0348, 9.39E-5, 1.0, 0.16, 0.18
    aH_min = aH*(1-0.2)
    aH_max = aH*(1+0.2)    

    from scipy import optimize
    #np.random.seed(42)

    nll = lambda *args: -log_likelihood(*args)
    initial = np.array([aH, C, m, scale, A, B]) #+ 0.001*np.random.randn(2)
    soln = optimize.minimize(nll, initial, method='Nelder-Mead', args=(ER_data, NR_data), options={'adaptive':True})
    aH_fit, C_fit, m_fit, scale_fit, A_fit, B_fit = soln.x
    
    #soln = optimize.brute(nll, ([0.0135, 0.018], [0.024, 0.048], [-0.0001, 0.0002], [0.9, 1.2], [0.06, 0.24], [0.06, 0.24]), args=(ER_data, NR_data))
    #soln = optimize.basinhopping(nll, initial, minimizer_kwargs={'args': (ER_data, NR_data)})
    #soln = optimize.shgo(nll, ([aH_min, aH_max], [0.024, 0.048], [-0.0001, 0.0002], [0.9, 1.2], [0.06, 0.24], [0.06, 0.24]), args=(ER_data, NR_data))
    #aH_fit, C_fit, m_fit, scale_fit, A_fit, B_fit = soln.x

    print (soln)
else:
    aH_fit, C_fit, m_fit, scale_fit, A_fit, B_fit = np.array([1.59555935e-02, 4.01182258e-02, 4.10754863e-05, 1.03491851e+00, 1.16818488e-01, 1.51974313e-01])

 final_simplex: (array([[ 6.81620597e-02, -1.18695031e-03,  4.63076262e-04,
        -7.35209968e-01,  3.54478429e-04, -5.27448902e-02],
       [ 6.93651818e-02, -1.22668514e-03,  4.66781464e-04,
        -7.35234247e-01,  3.46757042e-04, -5.64085017e-02],
       [ 7.24092588e-02, -1.18826817e-03,  4.64473365e-04,
        -7.35251604e-01,  4.12779363e-04, -5.70916160e-02],
       [ 7.11683647e-02, -9.35148922e-04,  4.61710197e-04,
        -7.35240219e-01,  4.07420060e-04, -5.42717795e-02],
       [ 7.44467061e-02, -1.15834817e-03,  4.62300319e-04,
        -7.35228786e-01,  4.07873684e-04, -5.70903595e-02],
       [ 7.32460168e-02, -1.03254476e-03,  4.62725716e-04,
        -7.35217836e-01,  3.66332418e-04, -5.65842791e-02],
       [ 7.75558591e-02, -1.25846212e-03,  4.65053855e-04,
        -7.35218176e-01,  4.81292213e-04, -6.14690326e-02]]), array([161259.8437807 , 161259.85644767, 161259.88996438, 161259.89105654,
       161259.93896117, 161259.93993884, 161259.95353682]))
           fu